# Notebook for getting PubMed metadata for FactPICO abstracts

**Metadata**: PMID, PMCID, title

In [ ]:
# import packages
from Bio import Entrez

Entrez.email = "yun.hy@northeastern.edu"

In [ ]:
# read original abstract csv file
